# Observe relationship GPP,LE, soil moist, sapflow and create my own f_str to plug into model

In [ ]:
# Settings
Username   = 'Beheerder'
years      = range(2001,2021)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath = os.path.join('../')
#print('datapath is set to %s'%datapath)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import plotly.express as px
#import cufflinks as cf
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
from matplotlib import cm
#from colorspacious import cspace_converter
import scipy.stats as stats
#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)

from datetime import datetime, timedelta
import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
from Loobos_Toolbox import dateparse, dateparse_Gapfilled, Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile
import matplotlib.dates as mdates

## Import soilmoist and sapflow data

In [ ]:
#these next two lines are to prevent re-loading the data. If you want to re-load data, instead comment them out
if not 'progress' in globals(): progress = list()
if not 'dataloaded' in progress:
  # Read files
    df_Comb         = Read_LoodatGapfill     (years,datapath)
    progress.append('dataloaded')

In [ ]:
#import sapflow
Sapf_data=pd.read_csv('Sapflow.csv')
Sapf_data['TIMESTAMP']=pd.to_datetime(Sapf_data['TIMESTAMP'])
Sapf_data.index=Sapf_data['TIMESTAMP']
Sapf_data.drop(columns='TIMESTAMP',inplace=True)
Sapf_data

In [ ]:
#import soil moist
SM_data=pd.read_csv('Soil_moisture.csv')
SM_data['datetime']=pd.to_datetime(SM_data['datetime'])
SM_data.index=SM_data['datetime']
SM_data.drop(columns='datetime',inplace=True)
SM_data

## End of Import soilmoist and sapflow data

## prep gpp data

In [ ]:
#import numpy as np
from FilterData import Filter_GPP_LE_NEE_VPD
#df_Comb_filter
df_Comb_filter=Filter_GPP_LE_NEE_VPD(df_Comb)

In [ ]:
df_Comb.columns

## Plot simple sens analysis GPP and LE vs soil moist and sapflow

In [ ]:

#join the two so that it's same len

df_Obs = df_Comb_filter.loc[:,['NEE','GPP_f','LE','VPD']]
df_GPP_SM = pd.concat([df_Obs,SM_data],axis=1,sort=False)
df_GPP_Sapf = pd.concat([df_Obs,Sapf_data],axis=1,sort=False)



In [ ]:
df_GPP_SM.dropna(subset='SM_l1',inplace=True)
df_GPP_SM

In [ ]:
df_GPP_Sapf.dropna(subset='Jt_mean',inplace=True)
df_GPP_Sapf

In [ ]:

fig,ax = plt.subplots()
#ax.xaxis.set_minor_locator(mdates.DayLocator())



ax.scatter(df_GPP_SM['SM_l1'],df_GPP_SM['LE']) #x,y

ax.set_ylabel('LE')
ax.set_xlabel('Soil moisture l1 (0-50cm)')


#ax.legend(loc=("upper left"))
#fig.autofmt_xdate()
plt.show()

In [ ]:

fig,ax = plt.subplots()
#ax.xaxis.set_minor_locator(mdates.DayLocator())



ax.scatter(df_GPP_SM['SM_l1'],df_GPP_SM['GPP_f']) #x,y

ax.set_ylabel('GPP')
ax.set_xlabel('Soil moisture l1 (0-50cm)')


#ax.legend(loc=("upper left"))
#fig.autofmt_xdate()
plt.show()

In [ ]:

fig,ax = plt.subplots()
#ax.xaxis.set_minor_locator(mdates.DayLocator())



ax.scatter(df_GPP_Sapf['Jt_mean'],df_GPP_Sapf['GPP_f']) #x,y

ax.set_ylabel('GPP')
ax.set_xlabel('Sap flow (cm3/cm2 s-1)')


#ax.legend(loc=("upper left"))
#fig.autofmt_xdate()
plt.show()

In [ ]:

fig,ax = plt.subplots()
#ax.xaxis.set_minor_locator(mdates.DayLocator())



ax.scatter(df_GPP_Sapf['Jt_mean'],df_GPP_Sapf['LE']) #x,y

ax.set_ylabel('LE')
ax.set_xlabel('Sap flow (cm3/cm2 s-1)')


#ax.legend(loc=("upper left"))
#fig.autofmt_xdate()
plt.show()

## end of plot simple sens analysis GPP and LE vs soil moist and sapflow

## plot proof of deep water simplified: sapflow during heatwave

In [ ]:
#2013 heatwave
fig,ax = plt.subplots()
#ax.xaxis.set_minor_locator(mdates.DayLocator())
st='2013-07-21'
ed='2013-07-27'

#the last recorded rain event was on 03-07-2013, the next rain event (Falls within the heatwave) is on 25-07-2013 
ax.plot(df_GPP_Sapf.loc[st:ed,'Jt_mean']) #x,y
twin1 = ax.twinx()
twin2 = ax.twinx()
twin1.plot(df_GPP_SM.loc[st:ed,'SM_l1'],c='purple') #x,y
twin1.plot(df_GPP_SM.loc[st:ed,'SM_l2'],c='red') #x,y
#twin2.plot(df_GPP_SM.loc[st:ed,'LE'],c='green')

ax.set_ylabel(r'Sap flow $(cm^{3} cm^{-2} s^{-1}$)')
twin1.set_ylabel(r'Soil moisture ($\Theta$) (%)')
twin2.spines["right"].set_position(("axes", 1.05))
twin2.set_ylabel('LE')
ax.set_xlabel('date')

#'2015-06-30':'2015-07-05'

#ax.legend(loc=("upper left"))
ax.grid()
fig.autofmt_xdate()
plt.show()

In [ ]:
import datetime as dt
plt.rcParams["figure.figsize"] = (20,6)

In [ ]:
#2013 heatwave + leadup
fig,ax = plt.subplots()
#ax.xaxis.set_minor_locator(mdates.DayLocator())
st='2013-07-05'
ed='2013-07-27'

#the last recorded rain event was on 03-07-2013, the next rain event (Falls within the heatwave) is on 25-07-2013 
ax.plot(df_GPP_Sapf.loc[st:ed,'Jt_mean']) #x,y
twin1 = ax.twinx()
twin2 = ax.twinx()
twin1.plot(df_GPP_SM.loc[st:ed,'SM_l1'],c='purple') #x,y
twin1.plot(df_GPP_SM.loc[st:ed,'SM_l2'],c='red') #x,y
#twin2.plot(df_GPP_SM.loc[st:ed,'LE'],c='green')

ax.set_ylabel(r'Sap flow $(cm^{3} cm^{-2} s^{-1}$)')
twin1.set_ylabel(r'Soil moisture ($\Theta$) (%)')
twin2.spines["right"].set_position(("axes", 1.05))
twin2.set_ylabel('LE')
ax.set_xlabel('date')
ax.axvline(dt.datetime(2013, 7, 21),c='orange')
ax.axvline(dt.datetime(2013, 7, 27),c='orange')

#'2015-06-30':'2015-07-05'

#ax.legend(loc=("upper left"))
ax.grid()
fig.autofmt_xdate()
plt.show()

In [ ]:
#2015 heatwave + leadup
fig,ax = plt.subplots()
#ax.xaxis.set_minor_locator(mdates.DayLocator())
st='2015-06-30'
ed='2015-07-05'

#the last recorded rain event was on 03-06-2013, the next rain event (Falls within the heatwave) is on 25-07-2013 
ax.plot(df_GPP_Sapf.loc[st:ed,'Jt_mean']) #x,y
twin1 = ax.twinx()
twin2 = ax.twinx()
twin1.plot(df_GPP_SM.loc[st:ed,'SM_l1'],c='purple') #x,y
twin1.plot(df_GPP_SM.loc[st:ed,'SM_l2'],c='red') #x,y
#twin2.plot(df_GPP_SM.loc[st:ed,'LE'],c='green')

ax.set_ylabel(r'Sap flow $(cm^{3} cm^{-2} s^{-1}$)')
twin1.set_ylabel(r'Soil moisture ($\Theta$) (%)')
twin2.spines["right"].set_position(("axes", 1.05))
twin2.set_ylabel('LE')
ax.set_xlabel('date')
ax.axvline(dt.datetime(2015, 6, 30),c='orange')
ax.axvline(dt.datetime(2015, 7, 5),c='orange')

#'2015-06-30':'2015-07-05'

#ax.legend(loc=("upper left"))
ax.grid()
fig.autofmt_xdate()
plt.show()

In [ ]:
#heatwave 2015
fig,ax = plt.subplots()
#ax.xaxis.set_minor_locator(mdates.DayLocator())
st='2015-06-30'
ed='2015-07-05'

#the last recorded rain event was on 03-07-2013, the next rain event (Falls within the heatwave) is on 25-07-2013 
ax.plot(df_GPP_Sapf.loc[st:ed,'Jt_mean']) #x,y
twin1 = ax.twinx()
twin2 = ax.twinx()
twin1.plot(df_GPP_SM.loc[st:ed,'SM_l1'],c='purple') #x,y
twin1.plot(df_GPP_SM.loc[st:ed,'SM_l2'],c='red') #x,y
#twin2.plot(df_GPP_SM.loc[st:ed,'LE'],c='green')

ax.set_ylabel(r'Sap flow $(cm^{3} cm^{-2} s^{-1}$)')
twin1.set_ylabel(r'Soil moisture ($\Theta$) (%)')
twin2.spines["right"].set_position(("axes", 1.05))
twin2.set_ylabel('LE')
ax.set_xlabel('date')

#'2015-06-30':'2015-07-05'

#ax.legend(loc=("upper left"))
ax.grid()
fig.autofmt_xdate()
plt.show()

In [ ]:
#heatwave 2015 + leadup
fig,ax = plt.subplots()
#ax.xaxis.set_minor_locator(mdates.DayLocator())
st='2015-06-24'
ed='2015-07-05'

#the last recorded rain event was on 03-07-2013, the next rain event (Falls within the heatwave) is on 25-07-2013 
ax.plot(df_GPP_Sapf.loc[st:ed,'Jt_mean']) #x,y
twin1 = ax.twinx()
twin2 = ax.twinx()
twin1.plot(df_GPP_SM.loc[st:ed,'SM_l1'],c='purple') #x,y
twin1.plot(df_GPP_SM.loc[st:ed,'SM_l2'],c='red') #x,y
#twin2.plot(df_GPP_SM.loc[st:ed,'LE'],c='green')

ax.set_ylabel(r'Sap flow $(cm^{3} cm^{-2} s^{-1}$)')
twin1.set_ylabel(r'Soil moisture ($\Theta$) (%)')
twin2.spines["right"].set_position(("axes", 1.05))
twin2.set_ylabel('LE')
ax.set_xlabel('date')

#'2015-06-30':'2015-07-05'

#ax.legend(loc=("upper left"))
ax.grid()
fig.autofmt_xdate()
plt.show()

## end of plot proof of deep water simplified: sapflow during heatwave